In [1]:
import jaydebeapi
import pandas as pd
import os
from config import ACCESS_TOKEN, UID

In [2]:


# Add JVM option to fix Apache Arrow memory access
os.environ['JAVA_TOOL_OPTIONS'] = '--add-opens=java.base/java.nio=ALL-UNNAMED'

# Connection parameters
JDBC_URL = "jdbc:databricks://adb-3055430050503041.1.azuredatabricks.net:443/default;transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/warehouses/6630450a2133788d;"
JDBC_DRIVER_PATH = "/Users/kiranbele/Documents/Projects/data-engineering-exploring-london-travel-network/Databricks-connection/DatabricksJDBC42-2.6.40.1071/DatabricksJDBC42.jar"

try:
    # Connect to Databricks
    conn = jaydebeapi.connect(
        "com.databricks.client.jdbc.Driver",
        JDBC_URL,
        {"PWD": ACCESS_TOKEN, "UID": UID},
        JDBC_DRIVER_PATH
    )

    # Execute query
    cursor = conn.cursor()
    query = "SELECT * FROM hive_metastore.default.tfl_journeys_final LIMIT 10"
    cursor.execute(query)

    # Fetch results and load into DataFrame
    results = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]  # Get column names
    df = pd.DataFrame(results, columns=columns)

    # Print results
    print("Query Result:")
    print(df)

    # Close resources
    cursor.close()
    conn.close()
    print("Connection closed.")
except Exception as e:
    print(f"Error: {e}")


Picked up JAVA_TOOL_OPTIONS: --add-opens=java.base/java.nio=ALL-UNNAMED


Query Result:
   month  year  days report_date       journey_type  journeys_millions
0      1  2010    31  2010-06-30  Underground & DLR          96.836391
1      2  2010    28  2010-06-30  Underground & DLR          90.330504
2      3  2010    31  2010-06-30  Underground & DLR          90.038014
3      4  2010    30  2010-09-30  Underground & DLR          92.544093
4      5  2010    31  2010-09-30  Underground & DLR          88.662911
5      6  2010    30  2010-09-30  Underground & DLR          86.375702
6      7  2010    31  2010-12-31  Underground & DLR          94.986035
7      8  2010    31  2010-12-31  Underground & DLR          97.024794
8      9  2010    30  2010-12-31  Underground & DLR          97.029640
9     10  2010    31  2011-03-31  Underground & DLR          77.369979
Connection closed.


## Had trouble using databricks-sql-connector package 
### since pyarrow and apache-arrow could not work together for some reason. 
